In [1]:
import pandas as pd
import numpy as np
import gymnasium as gym

In [2]:
df = pd.read_pickle("./data/raw/binance-BTCUSDT-5m.pkl")

In [3]:
df.tail()

,open,high,low,close,volume,date_close
date_open,,,,,,
2023-06-05 12:50:00,26693.72,26698.57,26666.00,26675.09,201.40100,2023-06-05 12:55:00
2023-06-05 12:55:00,26675.09,26691.23,26654.39,26690.76,230.29799,2023-06-05 13:00:00
2023-06-05 13:00:00,26690.76,26694.36,26640.00,26676.17,232.48809,2023-06-05 13:05:00
2023-06-05 13:05:00,26676.17,26689.98,26651.25,26689.98,136.69593,2023-06-05 13:10:00
2023-06-05 13:10:00,26689.97,26732.86,26689.97,26724.37,131.74040,2023-06-05 13:15:00


In [4]:
df["feature_close"] = df["close"]
df["feature_open"] = df["open"]
df["feature_high"] = df["high"]
df["feature_low"] = df["low"]
df["feature_volume"] = df["volume"]
df.dropna(inplace= True) # Clean your data !

In [5]:
import pandas_ta as ta

In [6]:
df.ta.indicators()

Pandas TA - Technical Analysis Indicators - v0.3.14b0
Total Indicators & Utilities: 205
Abbreviations:
    aberration, above, above_value, accbands, ad, adosc, adx, alma, amat, ao, aobv, apo, aroon, atr, bbands, below, below_value, bias, bop, brar, cci, cdl_pattern, cdl_z, cfo, cg, chop, cksp, cmf, cmo, coppock, cross, cross_value, cti, decay, decreasing, dema, dm, donchian, dpo, ebsw, efi, ema, entropy, eom, er, eri, fisher, fwma, ha, hilo, hl2, hlc3, hma, hwc, hwma, ichimoku, increasing, inertia, jma, kama, kc, kdj, kst, kurtosis, kvo, linreg, log_return, long_run, macd, mad, massi, mcgd, median, mfi, midpoint, midprice, mom, natr, nvi, obv, ohlc4, pdist, percent_return, pgo, ppo, psar, psl, pvi, pvo, pvol, pvr, pvt, pwma, qqe, qstick, quantile, rma, roc, rsi, rsx, rvgi, rvi, short_run, sinwma, skew, slope, sma, smi, squeeze, squeeze_pro, ssf, stc, stdev, stoch, stochrsi, supertrend, swma, t3, td_seq, tema, thermo, tos_stdevall, trima, trix, true_range, tsi, tsignals, ttm_trend, ui, 

In [7]:
df

,open,high,low,close,volume,date_close,feature_close,feature_open,feature_high,feature_low,feature_volume
date_open,,,,,,,,,,,
2018-12-31 16:05:00,3750.11,3752.26,3743.78,3748.61,73.484180,2018-12-31 16:10:00,3748.61,3750.11,3752.26,3743.78,73.484180
2018-12-31 16:10:00,3747.55,3753.95,3742.00,3745.71,49.545187,2018-12-31 16:15:00,3745.71,3747.55,3753.95,3742.00,49.545187
2018-12-31 16:15:00,3745.71,3747.13,3735.00,3739.62,151.770226,2018-12-31 16:20:00,3739.62,3745.71,3747.13,3735.00,151.770226
2018-12-31 16:20:00,3739.62,3745.00,3731.99,3742.30,107.421241,2018-12-31 16:25:00,3742.30,3739.62,3745.00,3731.99,107.421241
2018-12-31 16:25:00,3742.28,3745.00,3739.30,3741.95,43.780808,2018-12-31 16:30:00,3741.95,3742.28,3745.00,3739.30,43.780808
...,...,...,...,...,...,...,...,...,...,...,...
2023-06-05 12:50:00,26693.72,26698.57,26666.00,26675.09,201.401000,2023-06-05 12:55:00,26675.09,26693.72,26698.57,26666.00,201.401000
2023-06-05 12:55:00,26675.09,26691.23,26654.39,26690.76,230.297990,2023-06-05 13:00:00,26690.76,26675.09,26691.23,26654.39,230.297990
2023-06-05 13:00:00,26690.76,26694.36,26640.00,26676.17,232.488090,2023-06-05 13:05:00,26676.17,26690.76,26694.36,26640.00,232.488090


In [8]:
df.ta.sma(length=10, append=True)

date_open
2018-12-31 16:05:00          NaN
2018-12-31 16:10:00          NaN
2018-12-31 16:15:00          NaN
2018-12-31 16:20:00          NaN
2018-12-31 16:25:00          NaN
                         ...    
2023-06-05 12:50:00    26735.064
2023-06-05 12:55:00    26726.576
2023-06-05 13:00:00    26716.970
2023-06-05 13:05:00    26707.706
2023-06-05 13:10:00    26705.586
Name: SMA_10, Length: 464844, dtype: float64

In [10]:
df.ta.strategy(ta.CommonStrategy)

In [11]:
df

,open,high,low,close,volume,date_close,feature_close,feature_open,feature_high,feature_low,feature_volume,SMA_10,SMA_20,SMA_50,SMA_200,VOL_SMA_20
date_open,,,,,,,,,,,,,,,,
2018-12-31 16:05:00,3750.11,3752.26,3743.78,3748.61,73.484180,2018-12-31 16:10:00,3748.61,3750.11,3752.26,3743.78,73.484180,NaN,NaN,NaN,NaN,NaN
2018-12-31 16:10:00,3747.55,3753.95,3742.00,3745.71,49.545187,2018-12-31 16:15:00,3745.71,3747.55,3753.95,3742.00,49.545187,NaN,NaN,NaN,NaN,NaN
2018-12-31 16:15:00,3745.71,3747.13,3735.00,3739.62,151.770226,2018-12-31 16:20:00,3739.62,3745.71,3747.13,3735.00,151.770226,NaN,NaN,NaN,NaN,NaN
2018-12-31 16:20:00,3739.62,3745.00,3731.99,3742.30,107.421241,2018-12-31 16:25:00,3742.30,3739.62,3745.00,3731.99,107.421241,NaN,NaN,NaN,NaN,NaN
2018-12-31 16:25:00,3742.28,3745.00,3739.30,3741.95,43.780808,2018-12-31 16:30:00,3741.95,3742.28,3745.00,3739.30,43.780808,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-06-05 12:50:00,26693.72,26698.57,26666.00,26675.09,201.401000,2023-06-05 12:55:00,26675.09,26693.72,26698.57,26666.00,201.401000,26735.064,26754.6250,26764.1824,26943.14745,101.233168
2023-06-05 12:55:00,26675.09,26691.23,26654.39,26690.76,230.297990,2023-06-05 13:00:00,26690.76,26675.09,26691.23,26654.39,230.297990,26726.576,26751.5890,26761.5870,26940.39120,108.097298
2023-06-05 13:00:00,26690.76,26694.36,26640.00,26676.17,232.488090,2023-06-05 13:05:00,26676.17,26690.76,26694.36,26640.00,232.488090,26716.970,26747.1015,26759.1588,26937.54605,117.775819


In [6]:
def reward_function(history):
    return np.log(history["portfolio_valuation", -1] / history["portfolio_valuation", -2]) #log (p_t / p_t-1 )

In [7]:
from gym_trading_env.environments import TradingEnv
env = gym.make(
        "TradingEnv",
        name= "BTCUSD",
        df = df,
        windows= 15,
        positions = [ -1, -0.5, 0, 0.5, 1, 1.5, 2], # From -1 (=SHORT), to +1 (=LONG)
        initial_position = 'random', #Initial position
        trading_fees = 0.01/100, # 0.01% per stock buy / sell
        borrow_interest_rate= 0.0003/100, #per timestep (= 1h here)
        reward_function = reward_function,
        portfolio_initial_value = 1000, # in FIAT (here, USD)
        # max_episode_duration = 500,
    )

In [8]:
observation, info = env.reset()
observation

array([[ 3.7486101e+03,  3.7501101e+03,  3.7522600e+03,  3.7437800e+03,
         7.3484177e+01,  0.0000000e+00,  0.0000000e+00],
       [ 3.7457100e+03,  3.7475500e+03,  3.7539500e+03,  3.7420000e+03,
         4.9545189e+01,  0.0000000e+00,  0.0000000e+00],
       [ 3.7396201e+03,  3.7457100e+03,  3.7471299e+03,  3.7350000e+03,
         1.5177023e+02,  0.0000000e+00,  0.0000000e+00],
       [ 3.7423000e+03,  3.7396201e+03,  3.7450000e+03,  3.7319900e+03,
         1.0742124e+02,  0.0000000e+00,  0.0000000e+00],
       [ 3.7419500e+03,  3.7422800e+03,  3.7450000e+03,  3.7393000e+03,
         4.3780807e+01,  0.0000000e+00,  0.0000000e+00],
       [ 3.7433601e+03,  3.7431001e+03,  3.7440000e+03,  3.7370000e+03,
         7.6896660e+01,  0.0000000e+00,  0.0000000e+00],
       [ 3.7468501e+03,  3.7433701e+03,  3.7479900e+03,  3.7430200e+03,
         8.1525940e+01,  0.0000000e+00,  0.0000000e+00],
       [ 3.7434099e+03,  3.7456399e+03,  3.7468301e+03,  3.7402500e+03,
         9.1665771e+01,  

In [9]:
df

,open,high,low,close,volume,date_close,feature_close,feature_open,feature_high,feature_low,feature_volume
date_open,,,,,,,,,,,
2018-12-31 16:05:00,3750.11,3752.26,3743.78,3748.61,73.484180,2018-12-31 16:10:00,3748.61,3750.11,3752.26,3743.78,73.484180
2018-12-31 16:10:00,3747.55,3753.95,3742.00,3745.71,49.545187,2018-12-31 16:15:00,3745.71,3747.55,3753.95,3742.00,49.545187
2018-12-31 16:15:00,3745.71,3747.13,3735.00,3739.62,151.770226,2018-12-31 16:20:00,3739.62,3745.71,3747.13,3735.00,151.770226
2018-12-31 16:20:00,3739.62,3745.00,3731.99,3742.30,107.421241,2018-12-31 16:25:00,3742.30,3739.62,3745.00,3731.99,107.421241
2018-12-31 16:25:00,3742.28,3745.00,3739.30,3741.95,43.780808,2018-12-31 16:30:00,3741.95,3742.28,3745.00,3739.30,43.780808
...,...,...,...,...,...,...,...,...,...,...,...
2023-06-05 12:50:00,26693.72,26698.57,26666.00,26675.09,201.401000,2023-06-05 12:55:00,26675.09,26693.72,26698.57,26666.00,201.401000
2023-06-05 12:55:00,26675.09,26691.23,26654.39,26690.76,230.297990,2023-06-05 13:00:00,26690.76,26675.09,26691.23,26654.39,230.297990
2023-06-05 13:00:00,26690.76,26694.36,26640.00,26676.17,232.488090,2023-06-05 13:05:00,26676.17,26690.76,26694.36,26640.00,232.488090


In [10]:
from gym_trading_env.environments import TradingEnv
env2 = gym.make(
        "TradingEnv",
        name= "BTCUSD",
        df = df,
        windows= 15,
        positions = [ -1, -0.5, 0, 0.5, 1, 1.5, 2], # From -1 (=SHORT), to +1 (=LONG)
        initial_position = 'random', #Initial position
        trading_fees = 0.01/100, # 0.01% per stock buy / sell
        borrow_interest_rate= 0.0003/100, #per timestep (= 1h here)
        reward_function = reward_function,
        portfolio_initial_value = 1000, # in FIAT (here, USD)
        # max_episode_duration = 500,
    )
env2 = gym.wrappers.NormalizeObservation(env)


In [12]:
observation, info = env2.reset()
observation

array([[0.01      , 0.01      , 0.01      , 0.01      , 0.00999907,
        0.        , 0.        ],
       [0.01      , 0.01      , 0.01      , 0.01      , 0.00999796,
        0.        , 0.        ],
       [0.01      , 0.01      , 0.01      , 0.01      , 0.00999978,
        0.        , 0.        ],
       [0.01      , 0.01      , 0.01      , 0.01      , 0.00999957,
        0.        , 0.        ],
       [0.01      , 0.01      , 0.01      , 0.01      , 0.00999739,
        0.        , 0.        ],
       [0.01      , 0.01      , 0.01      , 0.01      , 0.00999915,
        0.        , 0.        ],
       [0.01      , 0.01      , 0.01      , 0.01      , 0.00999925,
        0.        , 0.        ],
       [0.01      , 0.01      , 0.01      , 0.01      , 0.0099994 ,
        0.        , 0.        ],
       [0.01      , 0.01      , 0.01      , 0.01      , 0.00999924,
        0.        , 0.        ],
       [0.01      , 0.01      , 0.01      , 0.01      , 0.00999976,
        0.        , 0. 